ASAP CRN Metadata validation - wave 2

# Team Scherzer. ASAP CRN Metadata validation - wave 2

28 Nov 2023
Andy Henrie




In [1]:
import pandas as pd

from pathlib import Path

from utils.io import read_meta_table, get_dtypes_dict, ReportCollector
from utils.qcutils import prep_table, validate_table, validate_file, read_ASAP_CDE, reorder_table_to_CDE



%load_ext autoreload
%autoreload 2

Streamlit NOT successfully


## Clean Tables
Some of the coding is wrong and easily fixed.  Some is not.  

TODO: share back details to Team Scherzer for better metadata

In [2]:
!pwd

/Users/ergonyc/Projects/ASAP/meta-clean


In [2]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/scherzer-credentials.json  



Activated service account credentials for: [raw-admin-scherzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [4]:

# !gcloud storage hash "gs://asap-raw-data-team-scherzer/**/*.gz"  --skip-crc32c --hex  --billing-project dnastack-asap-parkinsons > scherzer_hexhash.log

!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-scherzer/**/*.gz" > scherzer_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [3]:
!gsutil -u dnastack-asap-parkinsons cp -r "/Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023" "gs://asap-raw-data-team-scherzer/metadata/"

Copying file:///Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023/ASAP_CDE-ScherzerTeam.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
Copying file:///Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023/CLINPATH.csv [Content-Type=text/csv]...
Copying file:///Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023/SUBJECT.csv [Content-Type=text/csv]...
Copying file:///Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023/SAMPLE.csv [Content-Type=text/csv]...
- [4 files][540.2 KiB/540.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///Users/ergonyc/Projects/ASAP/team-scherzer/Scherzer_raw_meta11202023/DATA.csv [Content-T

In [4]:
# !gcloud storage ls -l "gs://asap-raw-data-team-scherzer/metadata"  --billing-project dnastack-asap-parkinsons 

     35161  2023-11-22T01:47:41Z  gs://asap-raw-data-team-scherzer/metadata/CLINPATH.csv
    595701  2023-11-22T01:47:41Z  gs://asap-raw-data-team-scherzer/metadata/DATA.csv
      1165  2023-11-22T01:47:40Z  gs://asap-raw-data-team-scherzer/metadata/PROTOCOL.csv
     32705  2023-11-22T01:47:40Z  gs://asap-raw-data-team-scherzer/metadata/SAMPLE.csv
      1684  2023-11-22T01:47:40Z  gs://asap-raw-data-team-scherzer/metadata/STUDY.csv
     15648  2023-11-22T01:47:40Z  gs://asap-raw-data-team-scherzer/metadata/SUBJECT.csv
                                  gs://asap-raw-data-team-scherzer/metadata/Scherzer_raw_meta11202023/
                                  gs://asap-raw-data-team-scherzer/metadata/v2/
TOTAL: 6 objects, 682064 bytes (666.08kiB)


## Team-Scherzer




In [68]:
## convert to seurat Object

## convert 
data_path = Path.home() / ("Projects/ASAP/team-scherzer")
metadata_path = data_path / "metadata"

metadata_version = "v2"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"


## convert 
data_path = Path.home() / ("Projects/ASAP/team-scherzer")
metadata_path = data_path / "Scherzer_raw_meta11202023"

SUBJECT = read_meta_table(metadata_path / "SUBJECT.csv", dtypes_dict)
CLINPATH = read_meta_table(metadata_path / "CLINPATH.csv", dtypes_dict)
STUDY = read_meta_table(metadata_path / "STUDY.csv", dtypes_dict)
PROTOCOL = read_meta_table(metadata_path / "PROTOCOL.csv", dtypes_dict)
SAMPLE = read_meta_table(metadata_path / "SAMPLE.csv", dtypes_dict)
DATA = read_meta_table(metadata_path / "DATA.csv", dtypes_dict)



# # excel_path = data_path / "ASAP_CDE_team_Scherzer.v11152023.xlsx"
# excel_path = data_path / "Scherzer_raw_meta11202023/ASAP_CDE-ScherzerTeam.xlsx"
# STUDY = pd.read_excel(excel_path,sheet_name="STUDY")#,header=1)#.drop(columns="Field")
# CLINPATH = pd.read_excel(excel_path,sheet_name="CLINPATH")#,header=1)#.drop(columns="Field")
# SUBJECT = pd.read_excel(excel_path,sheet_name="SUBJECT")#,header=1)#.drop(columns="Field")
# SAMPLE = pd.read_excel(excel_path,sheet_name="SAMPLE")#,header=1)#.drop(columns="Field")
# PROTOCOL = pd.read_excel(excel_path,sheet_name="PROTOCOL")#,header=1)#.drop(columns="Field")
# DATA = pd.read_excel(excel_path,sheet_name="DATA")#,header=1)#.drop(columns="Field")


read url


In [8]:
STUDY = prep_table(STUDY, CDE_df)


STUDY['metadata_version_date'] = METADATA_VERSION_DATE



In [24]:
STUDY = reorder_table_to_CDE(STUDY, "STUDY", CDE_df)

report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()


All required fields are present in *STUDY* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [25]:
PROTOCOL = reorder_table_to_CDE(PROTOCOL, "PROTOCOL", CDE_df)
report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()


All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [12]:
SUBJECT.head()

,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,BN0009,00-09,<NA>,<NA>,BSHRI,Human,Male,64,White,Unknown,...,yes,Unknown,unknown,Former smoker,30,33,Dementia,Unknown,Idiopathic PD,NaN
1,BN0329,03-29,<NA>,<NA>,BSHRI,Human,Male,79,White,Unknown,...,yes,Unknown,14,Unknown,Unknown,23,Dementia,Unknown,Idiopathic PD,NaN
2,BN0339,03-39,<NA>,<NA>,BSHRI,Human,Male,86,White,Unknown,...,no,Unknown,12,Former smoker,50,33,Normal,Unknown,Healthy Control,NaN
3,BN0341,03-41,<NA>,<NA>,BSHRI,Human,Male,89,White,Unknown,...,no,Unknown,16,Never,0,34,Normal,Unknown,Healthy Control,NaN
4,BN0347,03-47,<NA>,<NA>,BSHRI,Human,Female,95,White,Unknown,...,no,Unknown,12,Former smoker,Unknown,33,MCI,Unknown,Healthy Control,NaN


In [14]:
SUBJECT = prep_table(SUBJECT, CDE_df)

SUBJECT['hx_melanoma'] = SUBJECT['hx_melanoma'].replace({'Present':'Yes', 'Unknown': pd.NA})


In [15]:
SUBJECT['cognitive_status'] = SUBJECT['cognitive_status'].replace({'Unknown': pd.NA })

# SUBJECT.replace("Unknown", pd.NA, inplace=True)

# eductation levels 
SUBJECT['education_level'] = SUBJECT['education_level'].replace({'Unknown': pd.NA })

# map "Unknown" to pd.NA

# map ['14', '12', '16', '18', '13', '8', '15', '20', '19', '17', '11', '22', '10', '21'] ???

In [26]:
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE_df)
report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 94/94 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*hx_melanoma*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*education_level*_:  invalid values 💩'<NA>', '14', '12', '16', '18', '13', '8', '15', '20', '19', '17', '11', '22', '10', '21'
    - valid ➡️ 'High School', 'High School/GED', 'Some college without degree', 'Associate degree college', 'Bachelor's degree', 'Master's degree', 'Professional or doctoral degree', 'Refuse', 'Other'
- _*cognitive_status*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Normal', 'MCI', 'Dementia'



In [22]:
SAMPLE.head()

,sample_id,source_sample_id,cellranger_outs_folder,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,PD_BN0009_MTG_snRNAseq_rep1,<NA>,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,BN0009,rep1,1,0,batch1,Brain,Middle temporal gyrus,...,PATO:0000384 (male),<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
1,PD_BN0329_MTG_snRNAseq_rep1,<NA>,/data/neurogen/ASAP/scRNAseq/data/batch2_ASAP_...,BN0329,rep1,1,0,batch2,Brain,Middle temporal gyrus,...,PATO:0000384 (male),<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
2,HC_BN0339_MTG_snRNAseq_rep1,<NA>,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,BN0339,rep1,1,0,batch1,Brain,Middle temporal gyrus,...,PATO:0000384 (male),<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
3,HC_BN0341_MTG_snRNAseq_rep1,<NA>,/data/neurogen/ASAP/scRNAseq/data/batch1_ASAP_...,BN0341,rep1,1,0,batch1,Brain,Middle temporal gyrus,...,PATO:0000384 (male),<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>
4,HC_BN0347_MTG_snRNAseq_rep1,<NA>,/data/neurogen/ASAP/scRNAseq/data/batch2_ASAP_...,BN0347,rep1,1,0,batch2,Brain,Middle temporal gyrus,...,PATO:0000383 (female),<NA>,<NA>,<NA>,<NA>,EFO:0030003 '10x 3' v3 transcription profiling...,<NA>,NaN,NaN,<NA>


In [18]:
SAMPLE = prep_table(SAMPLE, CDE_df)

SAMPLE['region_level_1'] = SAMPLE['region_level_1'].replace({'Temporal Lobe':'Temporal lobe'})
SAMPLE['molecular_source'] = SAMPLE['molecular_source'].replace({'PolyA_RNA':'PolyA RNA'})


In [19]:
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"
subj_sex_mapper = dict(zip(SUBJECT['subject_id'], SUBJECT['sex']))
SAMPLE['sex_ontology_term_id'] = SAMPLE['subject_id'].map(subj_sex_mapper).replace({"Male":"PATO:0000384 (male)", "Female":"PATO:0000383 (female)" })


## ERROR in metadata

'source_sample_id' cannot be the same as 'subject_id'

In [20]:
SAMPLE['source_sample_id'] = pd.NA

In [27]:
SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE_df)

report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()

All required fields are present in *SAMPLE* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- DV200: 97/97 empty rows

	- pm_PH: 97/97 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*sequencing_length*_:  invalid values 💩'120'
    - valid ➡️ '25', '50', '100', '150'



In [28]:

CLINPATH = prep_table(CLINPATH, CDE_df)



In [29]:
# autopsy coding is too too idosyncratic.  leave 
# the path_nia_aa_a are coded as path_nia_ri... swap them 



# 'Low', 'Intermediate', 'High', 'None'
CLINPATH['path_nia_ri'] = CLINPATH['path_nia_aa_a'].replace({'Low ADNC': 'Low', 
                                                            'Not AD': 'None', 
                                                            'Intermediate ADNC': 'Intermediate'})
CLINPATH['path_nia_aa_a'] = pd.NA


# path_thal
CLINPATH['path_thal'].replace({'Phase 0 (A0)':'0', 
                                'Phase 1 (A1)':'1', 
                                'Phase 2 (A1)':'2', 
                                'Phase 3 (A2)':'3', 
                                'Phase 4 (A3)':'4',
                                'Phase 5 (A3)':'5', 
                                'Unknown':pd.NA, 
                                'Missing/unknown':pd.NA }, inplace=True)


CLINPATH['path_infarcs'].replace('Unknown', pd.NA, inplace=True)


# no idea what No, undx, yes maps to.  leave
CLINPATH['TDP43'].replace('Unknown', pd.NA, inplace=True)

# sn_neuronal_loss has alternate 0-3 coding.  leave

In [30]:
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)




In [31]:
# autopsy coding is too too idosyncratic.  leave 
# the path_nia_aa_a are coded as path_nia_ri... swap them 



# 'Low', 'Intermediate', 'High', 'None'
CLINPATH['path_nia_ri'] = CLINPATH['path_nia_aa_a'].replace({'Low ADNC': 'Low', 
                                                            'Not AD': 'None', 
                                                            'Intermediate ADNC': 'Intermediate'})
CLINPATH['path_nia_aa_a'] = pd.NA


# path_thal
CLINPATH['path_thal'].replace({'Phase 0 (A0)':'0', 
                                'Phase 1 (A1)':'1', 
                                'Phase 2 (A1)':'2', 
                                'Phase 3 (A2)':'3', 
                                'Phase 4 (A3)':'4',
                                'Phase 5 (A3)':'5', 
                                'Unknown':pd.NA, 
                                'Missing/unknown':pd.NA }, inplace=True)


CLINPATH['path_infarcs'].replace('Unknown', pd.NA, inplace=True)


# no idea what No, undx, yes maps to.  leave
CLINPATH['TDP43'].replace('Unknown', pd.NA, inplace=True)

# sn_neuronal_loss has alternate 0-3 coding.  leave

In [32]:

mckeith_map = {'l. Olfactory Bulb-Only':"Olfactory bulb only", 'lla. Brainstem Predominant':"Brainstem",
       'llb. Limbic Predominant':"Limbic (transitional)", 'lV. Neocortical':"Neocortical",
       'lll. Brainstem/Limbic':"Amygdala Predominant", '0. No Lewy bodies':"Absent", "Unknown":pd.NA}


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace(mckeith_map)

# leave path_nia_ri like this for now. not sure how to map "criteria not met" and "Not AD"

# leave amyloid_angiopathy_severity_scale like this for now. not sure how to map 'Cerebral amyloid angiopathy, temporal and occipital lobe','Cerebral amyloid angiopathy, frontal lobe']


In [33]:
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- path_year_death: 2/94 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_autopsy_dx_main*_:  invalid values 💩'PD/Dem', 'Control', 'Control (MCI)', 'PD/ND', '<NA>', 'Control, Non-motoric', 'PDD', 'MCI', 'PD (MCI)', 'Control (history)', 'Control (diseased)', 'MCI (clinical history)', 'Control (clinical history)'
    - valid ➡️ 'Lewy body disease nos', 'Parkinson's disease', 'Parkinson's disease with dementia', 'Dementia with Lewy bodies', 'Multiple system atrophy (SND>OPCA)', 'Multiple system atrophy (OPCA<SND)', 'Multiple system atrophy (SND=OPCA)', 'Progressive supranuclear palsy', 'Corticobasal degeneration', 'Globular glial tauoapathy (GGT)', 'Chronic traumatic encephalopathy (CTE)', 'FTLD-Tau (Pick's)', 'FTLD-Tau (MAPT)', 'FTLD-Tau (AGD)', 'FTLD-TDP43, Type A', 'FTLD-TDP43, Type B', 'FTLD-TDP43, Type C', 'FTLD-TDP43, Type D'

In [36]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_I1_001.fastq.gz,Index 1 Lane 1,bc037ea92d5359f3da1ba6da4f974e1c,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
1,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_I2_001.fastq.gz,Index 2 Lane 1,0c8ef844b76212e82428af03d341048b,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
2,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_R1_001.fastq.gz,Read 1 Lane 1,c6a41f29849152df2ba2391dc559149b,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
3,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_R2_001.fastq.gz,Read 2 Lane 1,4d610fdf5a72c8206b2f1505d8fcecd7,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
4,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L002_I1_001.fastq.gz,Index 1 Lane 2,e34917b3ec01cd09ea8b52c82db1dc83,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>


In [35]:
DATA_ = DATA.copy()
DATA = reorder_table_to_CDE(DATA, "DATA", CDE_df)

report = ReportCollector(destination="")
validate_table(DATA, "DATA", CDE_df, report)
report.print_log()

All required fields are present in *DATA* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- time: 1552/1552 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in DATA. 🥳



One last scrub of the columns 

In [37]:

# export_root = metadata_path / "clean"
# if not export_root.exists():
#     export_root.mkdir(parents=True, exist_ok=True)


export_root = Path.cwd() / "clean/team-Scherzer"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)

STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")
DATA.to_csv(export_root / "DATA.csv")


In [38]:
metadata_path = export_root

In [39]:
metadata_version = "v2"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"


## convert 

export_root

SUBJECT = read_meta_table(metadata_path / "SUBJECT.csv", dtypes_dict)
CLINPATH = read_meta_table(metadata_path / "CLINPATH.csv", dtypes_dict)
STUDY = read_meta_table(metadata_path / "STUDY.csv", dtypes_dict)
PROTOCOL = read_meta_table(metadata_path / "PROTOCOL.csv", dtypes_dict)
SAMPLE = read_meta_table(metadata_path / "SAMPLE.csv", dtypes_dict)
DATA = read_meta_table(metadata_path / "DATA.csv", dtypes_dict)

# SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv")
# CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")
# STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv")
# PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv")
# SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")
# DATA = pd.read_csv(f"{metadata_path}/DATA.csv")


read url


In [40]:
metadata_path

PosixPath('/Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer')

In [31]:
SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,PD_BN0009_MTG_snRNAseq_rep1,BN0009,<NA>,rep1,1,0,batch1,Brain,Middle temporal gyrus,Left,...,PATO:0000384 (male),Nan,Nan,Nan,Nan,EFO:0030003 '10x 3' v3 transcription profiling...,Nan,NaN,NaN,Nan
1,PD_BN0329_MTG_snRNAseq_rep1,BN0329,<NA>,rep1,1,0,batch2,Brain,Middle temporal gyrus,Left,...,PATO:0000384 (male),Nan,Nan,Nan,Nan,EFO:0030003 '10x 3' v3 transcription profiling...,Nan,NaN,NaN,Nan
2,HC_BN0339_MTG_snRNAseq_rep1,BN0339,<NA>,rep1,1,0,batch1,Brain,Middle temporal gyrus,Left,...,PATO:0000384 (male),Nan,Nan,Nan,Nan,EFO:0030003 '10x 3' v3 transcription profiling...,Nan,NaN,NaN,Nan
3,HC_BN0341_MTG_snRNAseq_rep1,BN0341,<NA>,rep1,1,0,batch1,Brain,Middle temporal gyrus,Left,...,PATO:0000384 (male),Nan,Nan,Nan,Nan,EFO:0030003 '10x 3' v3 transcription profiling...,Nan,NaN,NaN,Nan
4,HC_BN0347_MTG_snRNAseq_rep1,BN0347,<NA>,rep1,1,0,batch2,Brain,Middle temporal gyrus,Left,...,PATO:0000383 (female),Nan,Nan,Nan,Nan,EFO:0030003 '10x 3' v3 transcription profiling...,Nan,NaN,NaN,Nan


In [14]:
# 

# ## convert 
# data_path2 = Path.home() / ("Projects/ASAP/meta-clean/clean/team-Jakobsson")
# metadata_path2 = data_path2 / "v2_20231114"

# SUBJECT_ = read_meta_table(metadata_path2 / "SUBJECT.csv", dtypes_dict)
# CLINPATH_ = read_meta_table(metadata_path2 / "CLINPATH.csv", dtypes_dict)
# STUDY_ = read_meta_table(metadata_path2 / "STUDY.csv", dtypes_dict)
# PROTOCOL_ = read_meta_table(metadata_path2 / "PROTOCOL.csv", dtypes_dict)
# SAMPLE_ = read_meta_table(metadata_path2 / "SAMPLE.csv", dtypes_dict)
# DATA_ = read_meta_table(metadata_path2 / "DATA.csv", dtypes_dict)

In [26]:


# # fix SAMPLE replicate, replicate_count, batch with SAMPLE_

# batch_mapper = dict(zip(DATA['sample_id'], DATA['batch']))
                        
# batch = "batch_" + SAMPLE['sample_id'].map(batch_mapper)
# batch

# SAMPLE['batch']  = batch
# SAMPLE['replicate'] = 'rep1'
# SAMPLE['replicate_count'] = 1
# SAMPLE['repeated_sample'] = 0

# # also fix "replicate" coding in DATA "rep1"
# DATA['replicate'] = 'rep1'

In [41]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_I1_001.fastq.gz,Index 1 Lane 1,bc037ea92d5359f3da1ba6da4f974e1c,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
1,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_I2_001.fastq.gz,Index 2 Lane 1,0c8ef844b76212e82428af03d341048b,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
2,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_R1_001.fastq.gz,Read 1 Lane 1,c6a41f29849152df2ba2391dc559149b,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
3,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L001_R2_001.fastq.gz,Read 2 Lane 1,4d610fdf5a72c8206b2f1505d8fcecd7,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>
4,PD_BN0009_MTG_snRNAseq_rep1,rep1,1,0,batch1,fastq,PD_BN0009_MTG_snRNAseq_rep1_L002_I1_001.fastq.gz,Index 1 Lane 2,e34917b3ec01cd09ea8b52c82db1dc83,SN,RNA,Raw,Reads,NaN,<NA>,<NA>,<NA>


In [43]:
# checksum = DATA[['file_name','file_MD5']]
# checksum['check'] = checksum['file_name'].map(bucket_files_md5)
# checksum

In [44]:

from utils.checksums import extract_md5_from_details, extract_md5_from_details2,  extract_crc32c_from_details2, extract_hashes_from_gcloudstorage

In [45]:
source_hash = Path.home() / "Projects/ASAP/team-scherzer/scherzer_hash.log"

source_file_crc = extract_crc32c_from_details2(source_hash)
source_file_md5 = extract_md5_from_details2(source_hash)

source_hash.exists()

True

In [46]:
source_file_crc,source_file_md5 = extract_hashes_from_gcloudstorage(source_hash)

## crc32c hashes 

TODO: write hash tools for checking...

In [47]:
bucket_files_md5 = extract_md5_from_details2("scherzer_hexhash.log")


In [48]:
bucket_files_crc = extract_crc32c_from_details2("scherzer_hexhash.log")


In [49]:

checksum = DATA[['file_name','file_MD5']]


checksum['check_src_crc'] = checksum['file_name'].map(source_file_crc)
checksum['check_bucket_crc'] = checksum['file_name'].map(bucket_files_crc)


checksum['meta_MD5'] = checksum['file_MD5'].str.strip()
checksum['check_src_md5'] = checksum['file_name'].map(source_file_md5)
checksum['check_bucket_md5'] = checksum['file_name'].map(bucket_files_md5)




/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_16885/3456434893.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check_src_crc'] = checksum['file_name'].map(source_file_crc)
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_16885/3456434893.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check_bucket_crc'] = checksum['file_name'].map(bucket_files_crc)
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_16885/3456434893.py:8: SettingWithCopyWarning: 
A v

In [50]:
checksum

,file_name,file_MD5,check_src_crc,check_bucket_crc,meta_MD5,check_src_md5,check_bucket_md5
0,PD_BN0009_MTG_snRNAseq_rep1_L001_I1_001.fastq.gz,bc037ea92d5359f3da1ba6da4f974e1c,c99915b4,c99915b4,bc037ea92d5359f3da1ba6da4f974e1c,bc037ea92d5359f3da1ba6da4f974e1c,NaN
1,PD_BN0009_MTG_snRNAseq_rep1_L001_I2_001.fastq.gz,0c8ef844b76212e82428af03d341048b,f0af5e35,f0af5e35,0c8ef844b76212e82428af03d341048b,0c8ef844b76212e82428af03d341048b,NaN
2,PD_BN0009_MTG_snRNAseq_rep1_L001_R1_001.fastq.gz,c6a41f29849152df2ba2391dc559149b,151d9d97,151d9d97,c6a41f29849152df2ba2391dc559149b,c6a41f29849152df2ba2391dc559149b,NaN
3,PD_BN0009_MTG_snRNAseq_rep1_L001_R2_001.fastq.gz,4d610fdf5a72c8206b2f1505d8fcecd7,58bc244e,58bc244e,4d610fdf5a72c8206b2f1505d8fcecd7,4d610fdf5a72c8206b2f1505d8fcecd7,NaN
4,PD_BN0009_MTG_snRNAseq_rep1_L002_I1_001.fastq.gz,e34917b3ec01cd09ea8b52c82db1dc83,600d0703,600d0703,e34917b3ec01cd09ea8b52c82db1dc83,e34917b3ec01cd09ea8b52c82db1dc83,NaN
...,...,...,...,...,...,...,...
1547,HC_BN1762_MTG_snRNAseq_rep1_L003_R2_001.fastq.gz,887e3d000587b2774ce3cd7022078577,93e1775e,93e1775e,887e3d000587b2774ce3cd7022078577,887e3d000587b2774ce3cd7022078577,NaN
1548,HC_BN1762_MTG_snRNAseq_rep1_L004_I1_001.fastq.gz,44ac0c6d4f7f3b4502eead39564c8723,a361f850,a361f850,44ac0c6d4f7f3b4502eead39564c8723,44ac0c6d4f7f3b4502eead39564c8723,NaN
1549,HC_BN1762_MTG_snRNAseq_rep1_L004_I2_001.fastq.gz,dc114df490e1857072462ca090799005,b617330f,b617330f,dc114df490e1857072462ca090799005,dc114df490e1857072462ca090799005,NaN
1550,HC_BN1762_MTG_snRNAseq_rep1_L004_R1_001.fastq.gz,e24eddb1ac0b1f4d2bdd32b5f69dae27,e49ff1eb,e49ff1eb,e24eddb1ac0b1f4d2bdd32b5f69dae27,e24eddb1ac0b1f4d2bdd32b5f69dae27,NaN


transfer clean metadata


In [51]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/scherzer-credentials.json  



Activated service account credentials for: [raw-admin-scherzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [63]:
!gcloud storage ls -l "gs://asap-raw-data-team-scherzer/metadata/"  --billing-project dnastack-asap-parkinsons 

                                  gs://asap-raw-data-team-scherzer/metadata/Scherzer_raw_meta11202023/
                                  gs://asap-raw-data-team-scherzer/metadata/v2/
TOTAL: 0 objects, 0 bytes (0B)


In [53]:
!gcloud storage cp -r "/Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer" "gs://asap-raw-data-team-scherzer/metadata"  --billing-project dnastack-asap-parkinsons 

Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/CLINPATH.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/CLINPATH.csv
Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/SUBJECT.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/SUBJECT.csv
Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/SAMPLE.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/SAMPLE.csv
Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/DATA.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/DATA.csv
Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/STUDY.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/STUDY.csv
Copying file:///Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Scherzer/PROTOCOL.csv to gs://asap-raw-data-team-scherzer/metadata/team-Scherzer/PROTOCOL.csv
  Completed files 6/6 | 351.3kiB/351.3kiB                   

In [56]:
!gsutil -u dnastack-asap-parkinsons cp -r "./clean/team-Scherzer/*.csv"  "gs://asap-raw-data-team-scherzer/metadata/v2"


Copying file://./clean/team-Scherzer/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Scherzer/SUBJECT.csv [Content-Type=text/csv]...     
Copying file://./clean/team-Scherzer/SAMPLE.csv [Content-Type=text/csv]...      
Copying file://./clean/team-Scherzer/DATA.csv [Content-Type=text/csv]...        
- [4 files][348.6 KiB/348.6 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Scherzer/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Scherzer/PROTOCOL.csv [Content-Type=text/csv]...    
\ [6 files][351.3 KiB/351.3 KiB]                                                
Operation completed over 6 objects/351.3 KiB.                                    


In [70]:



# checksum_path = "scherzer_crc32c_checksums.csv"

# chksum = pd.read_csv(checksum_path)
# chksum.head()




,filename,crc32c_checksum
0,ILB_BN9947_MTG_snRNAseq_rep1_L004_R1_001.fastq.gz,904383552
1,ILB_BN9947_MTG_snRNAseq_rep1_L004_R2_001.fastq.gz,3515453253
2,HC_BN0855_MTG_snRNAseq_rep1_L001_I1_001.fastq.gz,4034171979
3,HC_BN0855_MTG_snRNAseq_rep1_L001_I2_001.fastq.gz,1461765174
4,HC_BN0855_MTG_snRNAseq_rep1_L001_R1_001.fastq.gz,2768035529


### copy to workflow-dev bucket

First copy each set of metadata locally ...

In [1]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [66]:
!gsutil cp "./clean/team-Scherzer/*.csv" "gs://asap-workflow-dev/metadata/v2/scherzer"

Copying file://./clean/team-Scherzer/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Scherzer/SUBJECT.csv [Content-Type=text/csv]...     
Copying file://./clean/team-Scherzer/SAMPLE.csv [Content-Type=text/csv]...      
Copying file://./clean/team-Scherzer/DATA.csv [Content-Type=text/csv]...        
- [4 files][348.6 KiB/348.6 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Scherzer/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Scherzer/PROTOCOL.csv [Content-Type=text/csv]...    
\ [6 files][351.3 KiB/351.3 KiB]                                                
Operation completed over 6 objects/351.3 KiB.                                    


In [7]:
!gsutil ls -al "gs://asap-workflow-dev/metadata/v2/lee"

      6243  2023-11-30T21:20:08Z  gs://asap-workflow-dev/metadata/v2/lee/CLINPATH.csv#1701379208316153  metageneration=1
     24214  2023-11-30T21:20:08Z  gs://asap-workflow-dev/metadata/v2/lee/DATA.csv#1701379208935791  metageneration=1
       968  2023-11-30T21:20:09Z  gs://asap-workflow-dev/metadata/v2/lee/PROTOCOL.csv#1701379209349939  metageneration=1
     23092  2023-11-30T21:20:08Z  gs://asap-workflow-dev/metadata/v2/lee/SAMPLE.csv#1701379208725066  metageneration=1
      1054  2023-11-30T21:20:09Z  gs://asap-workflow-dev/metadata/v2/lee/STUDY.csv#1701379209140495  metageneration=1
      4277  2023-11-30T21:20:08Z  gs://asap-workflow-dev/metadata/v2/lee/SUBJECT.csv#1701379208528070  metageneration=1
                                 gs://asap-workflow-dev/metadata/v2/lee/v2_20231128/
TOTAL: 6 objects, 59848 bytes (58.45 KiB)
